# 🎯 Supervised Learning - Interactive Interview Preparation

This notebook contains **runnable code examples** for all major supervised learning algorithms. Practice these implementations for your interviews!

## 📚 Contents:
1. [Linear Regression from Scratch](#linear-regression)
2. [Logistic Regression from Scratch](#logistic-regression) 
3. [Decision Trees from Scratch](#decision-trees)
4. [Model Evaluation & Metrics](#evaluation)
5. [Real Dataset Examples](#real-examples)
6. [Quick Interview Coding Challenges](#challenges)

In [ ]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression, load_iris, load_boston
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries imported successfully!")
print("🚀 Ready for supervised learning interview prep!")

---
## 📈 1. Linear Regression from Scratch

**🎯 Interview Question**: *"Implement linear regression from scratch using gradient descent"*

### Key Concepts to Remember:
- **Hypothesis**: h(x) = w₀ + w₁x₁ + w₂x₂ + ... + wₙxₙ
- **Cost Function**: J(w) = (1/2m) * Σ(h(x⁽ⁱ⁾) - y⁽ⁱ⁾)²
- **Gradient**: ∂J/∂w = (1/m) * X^T * (h(x) - y)

In [ ]:
class LinearRegressionScratch:
    """Linear Regression implemented from scratch using gradient descent"""
    
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.cost_history = []
    
    def fit(self, X, y):
        # Initialize parameters
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Forward pass: compute predictions
            y_pred = self.predict(X)
            
            # Compute cost (Mean Squared Error)
            cost = (1/(2*n_samples)) * np.sum((y_pred - y)**2)
            self.cost_history.append(cost)
            
            # Compute gradients
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
            db = (1/n_samples) * np.sum(y_pred - y)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    
    def plot_cost_history(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.cost_history, 'b-', linewidth=2)
        plt.title('Cost Function During Training', fontsize=16, fontweight='bold')
        plt.xlabel('Iterations', fontsize=12)
        plt.ylabel('Cost (MSE)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.show()

# Generate sample data for demonstration
X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegressionScratch(learning_rate=0.01, n_iterations=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🎯 Linear Regression Results:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Learned Weight: {model.weights[0]:.4f}")
print(f"Learned Bias: {model.bias:.4f}")

# Plot results
plt.figure(figsize=(15, 5))

# Plot 1: Data and predictions
plt.subplot(1, 3, 1)
plt.scatter(X_test, y_test, color='blue', alpha=0.6, label='Actual')
plt.scatter(X_test, y_pred, color='red', alpha=0.6, label='Predicted')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.xlabel('Feature')
plt.ylabel('Target')
plt.title('Predictions vs Actual')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Residuals
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

# Plot 3: Cost history
plt.subplot(1, 3, 3)
plt.plot(model.cost_history, 'b-', linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Cost (MSE)')
plt.title('Training Cost')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 🎯 2. Logistic Regression from Scratch

**🎯 Interview Question**: *"Implement logistic regression from scratch with gradient descent"*

### Key Concepts to Remember:
- **Sigmoid Function**: σ(z) = 1/(1 + e^(-z))
- **Cost Function**: J(w) = -(1/m) * Σ[y*log(h(x)) + (1-y)*log(1-h(x))]
- **Gradient**: ∂J/∂w = (1/m) * X^T * (h(x) - y)

In [ ]:
class LogisticRegressionScratch:
    """Logistic Regression implemented from scratch"""
    
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.cost_history = []
    
    def _add_intercept(self, X):
        """Add bias column to feature matrix"""
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def _sigmoid(self, z):
        """Sigmoid activation function"""
        # Clip z to prevent overflow
        z = np.clip(z, -250, 250)
        return 1 / (1 + np.exp(-z))
    
    def _cost_function(self, h, y):
        """Calculate logistic regression cost (cross-entropy)"""
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        # Add intercept term
        X = self._add_intercept(X)
        
        # Initialize weights
        self.weights = np.zeros(X.shape[1])
        
        # Gradient descent
        for i in range(self.n_iterations):
            # Forward pass
            z = np.dot(X, self.weights)
            h = self._sigmoid(z)
            
            # Calculate cost
            cost = self._cost_function(h, y)
            self.cost_history.append(cost)
            
            # Calculate gradient
            gradient = np.dot(X.T, (h - y)) / y.size
            
            # Update weights
            self.weights -= self.learning_rate * gradient
    
    def predict_proba(self, X):
        """Predict class probabilities"""
        X = self._add_intercept(X)
        return self._sigmoid(np.dot(X, self.weights))
    
    def predict(self, X, threshold=0.5):
        """Make binary predictions"""
        return (self.predict_proba(X) >= threshold).astype(int)

# Generate sample classification data
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                          n_informative=2, random_state=42, n_clusters_per_class=1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegressionScratch(learning_rate=0.1, n_iterations=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"🎯 Logistic Regression Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Visualization
plt.figure(figsize=(15, 5))

# Plot 1: Decision boundary
plt.subplot(1, 3, 1)
# Create a mesh for plotting decision boundary
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
mesh_points = np.c_[xx.ravel(), yy.ravel()]
Z = model.predict_proba(mesh_points)
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, levels=50, alpha=0.6, cmap='RdYlBu')
scatter = plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='RdYlBu', edgecolors='black')
plt.colorbar(scatter)
plt.title('Decision Boundary')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Plot 2: Confusion Matrix
plt.subplot(1, 3, 2)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Plot 3: Cost history
plt.subplot(1, 3, 3)
plt.plot(model.cost_history, 'b-', linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Cost (Cross-entropy)')
plt.title('Training Cost')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print classification report
print("\n📊 Detailed Classification Report:")
print(classification_report(y_test, y_pred))

---
## 🌳 3. Decision Tree from Scratch

**🎯 Interview Question**: *"Implement a decision tree classifier using Gini impurity"*

### Key Concepts to Remember:
- **Gini Impurity**: Gini = 1 - Σ(pᵢ)²
- **Information Gain**: IG = Gini_parent - Σ(wᵢ * Gini_child_i)
- **Splitting**: Choose feature and threshold that maximizes information gain

In [ ]:
from collections import Counter

class DecisionTreeScratch:
    """Decision Tree Classifier implemented from scratch"""
    
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None
    
    def _gini_impurity(self, y):
        """Calculate Gini impurity"""
        if len(y) == 0:
            return 0
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)
    
    def _information_gain(self, parent, left_child, right_child):
        """Calculate information gain"""
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        
        gain = (self._gini_impurity(parent) - 
                weight_left * self._gini_impurity(left_child) - 
                weight_right * self._gini_impurity(right_child))
        return gain
    
    def _best_split(self, X, y):
        """Find the best split for the data"""
        best_gain = -1
        best_feature = None
        best_threshold = None
        
        n_features = X.shape[1]
        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])
            
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                
                if (len(y[left_mask]) < self.min_samples_leaf or 
                    len(y[right_mask]) < self.min_samples_leaf):
                    continue
                
                gain = self._information_gain(y, y[left_mask], y[right_mask])
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_gain
    
    def _build_tree(self, X, y, depth=0):
        """Recursively build the decision tree"""
        # Stopping criteria
        if (depth >= self.max_depth or 
            len(set(y)) == 1 or 
            len(y) < self.min_samples_split):
            # Return leaf node with majority class
            return Counter(y).most_common(1)[0][0]
        
        # Find best split
        feature_idx, threshold, gain = self._best_split(X, y)
        
        if gain == 0:
            return Counter(y).most_common(1)[0][0]
        
        # Split the data
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        
        # Create node
        node = {
            'feature': feature_idx,
            'threshold': threshold,
            'gain': gain,
            'samples': len(y),
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }
        
        return node
    
    def fit(self, X, y):
        """Train the decision tree"""
        self.root = self._build_tree(X, y)
    
    def _predict_sample(self, sample, node):
        """Predict a single sample"""
        if not isinstance(node, dict):
            return node
        
        if sample[node['feature']] <= node['threshold']:
            return self._predict_sample(sample, node['left'])
        else:
            return self._predict_sample(sample, node['right'])
    
    def predict(self, X):
        """Make predictions on new data"""
        return np.array([self._predict_sample(sample, self.root) for sample in X])
    
    def _get_tree_depth(self, node):
        """Calculate tree depth"""
        if not isinstance(node, dict):
            return 0
        return 1 + max(self._get_tree_depth(node['left']), 
                      self._get_tree_depth(node['right']))
    
    def print_tree(self, node=None, depth=0):
        """Print the tree structure"""
        if node is None:
            node = self.root
        
        if not isinstance(node, dict):
            print(f"{'  ' * depth}📋 Predict: Class {node}")
            return
        
        print(f"{'  ' * depth}🌿 Feature {node['feature']} <= {node['threshold']:.2f} (gain: {node['gain']:.3f}, samples: {node['samples']})")
        print(f"{'  ' * depth}├── True:")
        self.print_tree(node['left'], depth + 1)
        print(f"{'  ' * depth}└── False:")
        self.print_tree(node['right'], depth + 1)

# Generate sample data
X, y = make_classification(n_samples=300, n_features=4, n_redundant=0, 
                          n_informative=4, random_state=42, n_classes=3)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
dt = DecisionTreeScratch(max_depth=4, min_samples_split=5, min_samples_leaf=2)
dt.fit(X_train, y_train)

# Make predictions
y_pred = dt.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Decision Tree Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Tree Depth: {dt._get_tree_depth(dt.root)}")

# Print tree structure
print("\n🌳 Tree Structure:")
dt.print_tree()

# Visualizations
plt.figure(figsize=(15, 5))

# Plot 1: Feature importance (based on first split)
plt.subplot(1, 3, 1)
feature_names = [f'Feature {i}' for i in range(X.shape[1])]
# Simple feature importance based on root split
importance = np.zeros(X.shape[1])
if isinstance(dt.root, dict):
    importance[dt.root['feature']] = dt.root['gain']
plt.bar(feature_names, importance)
plt.title('Feature Importance (Root Split)')
plt.xlabel('Features')
plt.ylabel('Information Gain')
plt.xticks(rotation=45)

# Plot 2: Confusion Matrix
plt.subplot(1, 3, 2)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Plot 3: Class distribution
plt.subplot(1, 3, 3)
unique, counts = np.unique(y_train, return_counts=True)
plt.bar([f'Class {c}' for c in unique], counts, alpha=0.7)
plt.title('Class Distribution in Training Data')
plt.xlabel('Classes')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

print("\n📊 Detailed Classification Report:")
print(classification_report(y_test, y_pred))

---
## 📊 4. Model Evaluation & Metrics

**🎯 Interview Question**: *"How do you evaluate a classification/regression model? Write code to compute all relevant metrics."*

In [ ]:
def comprehensive_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate all classification metrics"""
    from sklearn.metrics import (
        accuracy_score, precision_score, recall_score, f1_score,
        confusion_matrix, classification_report, roc_auc_score, 
        precision_recall_curve, roc_curve
    )
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # Confusion matrix components
    cm = confusion_matrix(y_true, y_pred)
    
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm
    }
    
    # AUC-ROC for binary classification
    if len(np.unique(y_true)) == 2 and y_pred_proba is not None:
        auc_roc = roc_auc_score(y_true, y_pred_proba)
        results['auc_roc'] = auc_roc
    
    return results

def comprehensive_regression_metrics(y_true, y_pred):
    """Calculate all regression metrics"""
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    # Mean Absolute Percentage Error
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2_score': r2,
        'mape': mape
    }

def plot_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """Plot comprehensive classification evaluation"""
    from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0])
    axes[0,0].set_title('Confusion Matrix')
    axes[0,0].set_xlabel('Predicted')
    axes[0,0].set_ylabel('Actual')
    
    # Class distribution
    unique, counts = np.unique(y_true, return_counts=True)
    axes[0,1].bar([f'Class {c}' for c in unique], counts, alpha=0.7)
    axes[0,1].set_title('True Class Distribution')
    axes[0,1].set_xlabel('Classes')
    axes[0,1].set_ylabel('Count')
    
    if y_pred_proba is not None and len(np.unique(y_true)) == 2:
        # ROC Curve
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        auc_score = roc_auc_score(y_true, y_pred_proba)
        axes[1,0].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.3f})')
        axes[1,0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
        axes[1,0].set_xlabel('False Positive Rate')
        axes[1,0].set_ylabel('True Positive Rate')
        axes[1,0].set_title('ROC Curve')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
        
        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
        axes[1,1].plot(recall, precision, linewidth=2)
        axes[1,1].set_xlabel('Recall')
        axes[1,1].set_ylabel('Precision')
        axes[1,1].set_title('Precision-Recall Curve')
        axes[1,1].grid(True, alpha=0.3)
    else:
        # Prediction vs True labels
        axes[1,0].scatter(y_true, y_pred, alpha=0.6)
        axes[1,0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
        axes[1,0].set_xlabel('True Labels')
        axes[1,0].set_ylabel('Predicted Labels')
        axes[1,0].set_title('Predictions vs True Labels')
        
        # Remove empty subplot
        fig.delaxes(axes[1,1])
    
    plt.tight_layout()
    plt.show()

def plot_regression_metrics(y_true, y_pred):
    """Plot comprehensive regression evaluation"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Predictions vs Actual
    axes[0,0].scatter(y_true, y_pred, alpha=0.6)
    axes[0,0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    axes[0,0].set_xlabel('True Values')
    axes[0,0].set_ylabel('Predicted Values')
    axes[0,0].set_title('Predictions vs True Values')
    axes[0,0].grid(True, alpha=0.3)
    
    # Residuals plot
    residuals = y_true - y_pred
    axes[0,1].scatter(y_pred, residuals, alpha=0.6)
    axes[0,1].axhline(y=0, color='red', linestyle='--')
    axes[0,1].set_xlabel('Predicted Values')
    axes[0,1].set_ylabel('Residuals')
    axes[0,1].set_title('Residual Plot')
    axes[0,1].grid(True, alpha=0.3)
    
    # Histogram of residuals
    axes[1,0].hist(residuals, bins=20, alpha=0.7, edgecolor='black')
    axes[1,0].axvline(x=0, color='red', linestyle='--')
    axes[1,0].set_xlabel('Residuals')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Residuals')
    axes[1,0].grid(True, alpha=0.3)
    
    # Q-Q plot (approximate)
    from scipy import stats
    stats.probplot(residuals, dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot of Residuals')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Demo with logistic regression
print("🎯 Classification Metrics Demo")
print("=" * 50)

# Generate binary classification data
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                          n_informative=2, random_state=42, n_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression
log_reg = LogisticRegressionScratch(learning_rate=0.1, n_iterations=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)

# Calculate metrics
class_metrics = comprehensive_classification_metrics(y_test, y_pred, y_pred_proba)

print(f"Accuracy: {class_metrics['accuracy']:.4f}")
print(f"Precision: {class_metrics['precision']:.4f}")
print(f"Recall: {class_metrics['recall']:.4f}")
print(f"F1-Score: {class_metrics['f1_score']:.4f}")
print(f"AUC-ROC: {class_metrics['auc_roc']:.4f}")

# Plot classification metrics
plot_classification_metrics(y_test, y_pred, y_pred_proba)

print("\n🎯 Regression Metrics Demo")
print("=" * 50)

# Generate regression data
X_reg, y_reg = make_regression(n_samples=1000, n_features=5, noise=10, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Train linear regression
lin_reg = LinearRegressionScratch(learning_rate=0.01, n_iterations=1000)
lin_reg.fit(X_train_reg, y_train_reg)

y_pred_reg = lin_reg.predict(X_test_reg)

# Calculate metrics
reg_metrics = comprehensive_regression_metrics(y_test_reg, y_pred_reg)

print(f"MAE: {reg_metrics['mae']:.4f}")
print(f"MSE: {reg_metrics['mse']:.4f}")
print(f"RMSE: {reg_metrics['rmse']:.4f}")
print(f"R² Score: {reg_metrics['r2_score']:.4f}")
print(f"MAPE: {reg_metrics['mape']:.2f}%")

# Plot regression metrics
plot_regression_metrics(y_test_reg, y_pred_reg)

---
## 🎯 5. Quick Interview Coding Challenges

Practice these common interview questions!

In [ ]:
print("🔥 INTERVIEW CODING CHALLENGES")
print("=" * 50)

# Challenge 1: Implement F1 Score from scratch
def f1_score_from_scratch(y_true, y_pred):
    """Calculate F1 score from confusion matrix"""
    # Calculate confusion matrix manually
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, precision, recall

# Test F1 score implementation
y_true_test = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0])
y_pred_test = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 0])

f1_manual, prec_manual, rec_manual = f1_score_from_scratch(y_true_test, y_pred_test)
f1_sklearn = f1_score(y_true_test, y_pred_test)

print(f"Challenge 1 - F1 Score Implementation:")
print(f"Manual F1: {f1_manual:.4f}")
print(f"sklearn F1: {f1_sklearn:.4f}")
print(f"✅ Match: {abs(f1_manual - f1_sklearn) < 1e-6}")

# Challenge 2: Implement train-test split
def train_test_split_scratch(X, y, test_size=0.2, random_state=None):
    """Implement train-test split from scratch"""
    if random_state:
        np.random.seed(random_state)
    
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    
    # Generate random indices
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Test train-test split
X_test_data = np.random.randn(100, 3)
y_test_data = np.random.randint(0, 2, 100)

X_train_manual, X_test_manual, y_train_manual, y_test_manual = train_test_split_scratch(
    X_test_data, y_test_data, test_size=0.2, random_state=42
)

print(f"\nChallenge 2 - Train-Test Split:")
print(f"Original data shape: {X_test_data.shape}")
print(f"Training set shape: {X_train_manual.shape}")
print(f"Test set shape: {X_test_manual.shape}")
print(f"✅ Split ratio: {len(X_test_manual) / len(X_test_data):.1f}")

# Challenge 3: Implement K-Fold Cross Validation
def kfold_cross_validation(X, y, k=5, random_state=None):
    """Implement K-Fold cross validation from scratch"""
    if random_state:
        np.random.seed(random_state)
    
    n_samples = len(X)
    indices = np.random.permutation(n_samples)
    fold_size = n_samples // k
    
    folds = []
    for i in range(k):
        start_idx = i * fold_size
        end_idx = start_idx + fold_size if i < k-1 else n_samples
        
        test_indices = indices[start_idx:end_idx]
        train_indices = np.concatenate([indices[:start_idx], indices[end_idx:]])
        
        folds.append({
            'train_indices': train_indices,
            'test_indices': test_indices,
            'X_train': X[train_indices],
            'X_test': X[test_indices],
            'y_train': y[train_indices],
            'y_test': y[test_indices]
        })
    
    return folds

# Test K-Fold implementation
folds = kfold_cross_validation(X_test_data, y_test_data, k=5, random_state=42)

print(f"\nChallenge 3 - K-Fold Cross Validation:")
print(f"Number of folds: {len(folds)}")
for i, fold in enumerate(folds):
    print(f"Fold {i+1}: Train={len(fold['train_indices'])}, Test={len(fold['test_indices'])}")

# Challenge 4: Implement Standard Scaler
class StandardScalerScratch:
    """Standard Scaler implemented from scratch"""
    
    def __init__(self):
        self.mean_ = None
        self.std_ = None
    
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.mean_) / self.std_
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)
    
    def inverse_transform(self, X):
        return X * self.std_ + self.mean_

# Test Standard Scaler
scaler_manual = StandardScalerScratch()
X_scaled_manual = scaler_manual.fit_transform(X_test_data)

from sklearn.preprocessing import StandardScaler
scaler_sklearn = StandardScaler()
X_scaled_sklearn = scaler_sklearn.fit_transform(X_test_data)

print(f"\nChallenge 4 - Standard Scaler:")
print(f"Original mean: {np.mean(X_test_data, axis=0)[:3]}...") 
print(f"Manual scaled mean: {np.mean(X_scaled_manual, axis=0)[:3]}...")
print(f"sklearn scaled mean: {np.mean(X_scaled_sklearn, axis=0)[:3]}...")
print(f"✅ Match: {np.allclose(X_scaled_manual, X_scaled_sklearn)}")

print("\n🎉 All challenges completed! You're ready for the interview! 🎉")

---
## 📝 Quick Reference Cheat Sheet

### 🔥 Must Remember Formulas:

**Linear Regression:**
- Hypothesis: `ŷ = w^T * X + b`
- Cost: `J = (1/2m) * Σ(ŷ - y)²`
- Gradient: `∂J/∂w = (1/m) * X^T * (ŷ - y)`

**Logistic Regression:**
- Sigmoid: `σ(z) = 1/(1 + e^(-z))`
- Cost: `J = -(1/m) * Σ[y*log(ŷ) + (1-y)*log(1-ŷ)]`
- Gradient: `∂J/∂w = (1/m) * X^T * (ŷ - y)`

**Metrics:**
- Precision: `TP/(TP+FP)`
- Recall: `TP/(TP+FN)`
- F1-Score: `2*(Precision*Recall)/(Precision+Recall)`
- Accuracy: `(TP+TN)/(TP+TN+FP+FN)`

### 🎯 Interview Tips:
1. **Always ask clarifying questions** about the problem
2. **Start with a simple baseline** before complex models
3. **Explain your thought process** while coding
4. **Consider edge cases** (empty data, single class, etc.)
5. **Think about scalability** and computational complexity
6. **Validate your implementation** with known libraries

### 🚀 Common Gotchas:
- Don't forget to scale features for distance-based algorithms
- Use stratified sampling for imbalanced datasets
- Never use test data for hyperparameter tuning
- Check for data leakage in time series problems
- Handle missing values appropriately
- Consider class imbalance in evaluation metrics

---
**🎉 You're now ready to ace your supervised learning interviews! Practice these implementations and understand the underlying concepts. Good luck! 🍀**